### Import stuff

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedShuffleSplit
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.neural_network import MLPClassifier, BernoulliRBM
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn import linear_model as sklean_lms
from sklearn.naive_bayes import GaussianNB

### Approach 1

In [13]:
def read_data(filename):
    return pd.read_csv(filename) 

def get_tfidf_vectorizer(sentences):
    _m = TfidfVectorizer(max_df=0.5, max_features = 5000,
                                 min_df=2, stop_words='english',
                                 use_idf=True)
    _m.fit(sentences)
    return _m

def get_range_tfidf_vectorizer(sentences):
    _m = TfidfVectorizer(max_df=0.5, ngram_range=(2,3), max_features = 5000,
                                 min_df=1, stop_words='english',
                                 use_idf=True)
    _m.fit(sentences)
    return _m

def get_vectors(m1, m2, sentences):
    x1 = m1.transform(sentences)
    x2 = m2.transform(sentences)
    return sp.hstack([x1, x2])

class ExtendedMultiOutputClassifier(MultiOutputClassifier):
    def transform(self, X):
        """
        Add a transform method to the classifier because it is mandatory for steps of a pipeline
        to provide fit and transform methods.
        """
        # for RF
        _o = self.predict_proba(X)
        return np.concatenate(_o, axis=1)
        # for SVM
        #return self.predict(X)

In [39]:
INDATA_LOCATION = 'C:/Users/sharm/Desktop/Dat5Melb/Final_Project/Datasets/train/train_new.csv'

# utility definitions for easier handling of the dataset column names
TEXT_COLUMN = 'comment_text'
CLASS_TOXIC, CLASS_SEVER_TOXIC, CLASS_OBSCENE, CLASS_THREAT, CLASS_INSULT, \
    CLASS_IDENTITY_HATE = ["toxic", "severe_toxic", "obscene", "threat", \
                           "insult", "identity_hate"]
CLASSES = [CLASS_TOXIC, CLASS_SEVER_TOXIC, CLASS_OBSCENE, CLASS_THREAT, CLASS_INSULT, CLASS_IDENTITY_HATE]

# read the comments and associated classification data 
dataDf = read_data(INDATA_LOCATION)
print(dataDf.shape)    

(156658, 8)


In [40]:
perf = []

## shuffle and split the dataset stratified by the number of classifications of a data point
## for balancing across resulting modeling and evaluation datasets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=0)
for train_index, test_index in sss.split(np.zeros(len(dataDf)), dataDf[CLASSES].sum(axis=1)):
    pass
dataDf_modeling = dataDf.iloc[train_index]
dataDf_testing = dataDf.iloc[test_index]

## lets create one doc2vec model from everything
print('\tBuilding model')
model1 = get_tfidf_vectorizer(dataDf_modeling[TEXT_COLUMN])
model2 = get_range_tfidf_vectorizer(dataDf_modeling[TEXT_COLUMN])

## lets prepare the data vectors
#X_orig = model.transform(dataDf_modeling[TEXT_COLUMN])
## lets reduce the dims
print('\tMaking vectors')
X = get_vectors(model1, model2, dataDf_modeling[TEXT_COLUMN])

	Building model
	Making vectors


In [41]:
## learn RFs
print('\tBuilding RFs')
moc = ExtendedMultiOutputClassifier(RandomForestClassifier(n_estimators = 100, n_jobs=8))
#moc = ExtendedMultiOutputClassifier(SVC(kernel='linear', probability=True))
moc.fit(X, dataDf_modeling[CLASSES])

	Building RFs


ExtendedMultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=8,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
               n_jobs=1)

In [42]:
## learn MLP
print('\tBuilding NN')
mlp = MLPClassifier(solver='sgd', activation='logistic', learning_rate='adaptive', momentum=0.9, alpha=1e-6, hidden_layer_sizes=(100, 100), random_state=1, tol=1e-15)
mlp.fit(moc.transform(X), dataDf_modeling[CLASSES])

	Building NN


MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=1e-15, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [43]:
## predict on test set
print('\tPrepare test vectors')
X_test = get_vectors(model1, model2, dataDf_testing[TEXT_COLUMN])

print('\tPredicting classes')
predicted = mlp.predict_proba(moc.transform(X_test))

predictedDf = pd.DataFrame(predicted)
predictedDf.columns = CLASSES

print('\tEvaluating')
# mean auc
aucs = map(lambda klass: metrics.roc_auc_score(dataDf_testing[klass], predictedDf[klass]), CLASSES)
# MSE
d = predicted - dataDf_testing[CLASSES]
sq_difs = map(lambda x: np.dot(x, x.T), d.as_matrix())

print('\tMean AUC: %f' %np.mean(aucs))
print('MSE: %f' %(np.sum(sq_difs) * 1.0 / len(d)))

	Prepare test vectors
	Predicting classes
	Evaluating
	Mean AUC: 0.926226
MSE: 0.075917


In [44]:
# if False:
TESTDATA_LOCATION = 'C:/Users/sharm/Desktop/Dat5Melb/Final_Project/Datasets/test1/test.csv'
testDf = pd.read_csv(TESTDATA_LOCATION)

X_sub = get_vectors(model1, model2, testDf[TEXT_COLUMN])
y_sub = pd.DataFrame(mlp.predict_proba(moc.transform(X_sub)))
y_sub.columns = CLASSES

subDf = pd.concat([testDf['id'], y_sub], axis=1)

subDf.head()
subDf.to_csv('C:/Users/sharm/Desktop/Dat5Melb/Final_Project/submission.csv', index=False)